<a href="https://colab.research.google.com/github/miel123/DSCapstone/blob/Coursera-Data-Science-Capstone/Capstone_Project_The_Battle_of_Neighborhoods_(Week_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size = 5>The Battle of Neighborhoods</font></h1>


---
---- 



###Table of content

 [1. Introduction: Business Problem](#introduction)

 [2. Data](#Data)

 [3. Methodology](#Methodology)

 [4. Analysis](#Analysis)

 [5. Conclusion](#Conclusion)



## 1. Business Problem <a name="introduction"></a>
Covid-19 is conquering this world with uncertainty. However, geographical components of the world is not affected. While food industry is hurt by the pandemic, hospitals are needed to strive themselves to cover the increasing number of victims. Thus, the current number of hospitals is not enough, hence, they have to face more and more pressure by Covid-19. Under limited budget, gorvernment have to build more hospitals to secure people. </n>

The problem here is to find where to build new hospital(s) to efficiently handle every patient in Toronto, USA.

The problem is, where is the most suitable to build a new one. In this lab, let's leverage data analytics to solve it. 

**Solution:** 
- Eliminate locations with large number of hospitals 
- Locations must be near highly populated areas

## 2. Data <a name="Data"></a>
Based on the defined problem, factors affect the decision would be:
1. Number of existing hospitals and there location in the neighborhood
3. Ratio of hospitals/population in each neighborhood (if possible)


Thus, steps and data needed is:
- Round neiborhoods with low number of hospitals in Toronto
- Create heatmap of population in Tokyo in rounded area (if possible)
- Use K-mean cluster to pick out areas with large number of population and low number of hospitals. 
- The clustered area are candidates locations for building a new hospital.




## Methodology <a name="Methodology"></a>

Get location coordinates

 Because just extracing address in Tokyo geogegraphically may not cover the population of hospitals, we can approach by rounding from the city center 

In [1]:
#@title Set up 
!pip install pyproj
import pyproj #a coordinates library
import shapely.geometry
import folium
import pandas as pd
import numpy as np
!pip install bs4
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


     |████████████████████████████████| 6.6MB 13.3MB/s 


In [2]:
#@title Wep scrapping
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(link).text
soup = BeautifulSoup(html_data,'html5lib')
tables = soup.find_all('table')         
tables[0]
table_content = []
for row in tables[0].find_all('td'):
  cell = {}
  if row.span.text == 'Not assigned':
    pass
  else:
      cell['PostalCode'] = row.p.text[:3]
      cell['Borough'] = (row.span.text).split('(')[0]
      cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' / ',',')).replace(')','')).strip(' ')
      table_content.append(cell)

pd.DataFrame(table_content)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South,King's Mill Park,Sunnylea,Humbe..."


In [3]:
df = pd.DataFrame(table_content)


We now have the neiborhoods name without location, let's add them from the available dataset

In [4]:
#@title Fetching Coordinates
from google.colab import files
lonlat = files.upload()
import io
coord = pd.read_csv(io.BytesIO(lonlat['Geospatial_Coordinates.csv']))
df_coord = df.merge(coord, 
              left_on = 'PostalCode',
              right_on = 'Postal Code',
              how = 'left')
df_coord

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South,King's Mill Park,Sunnylea,Humbe...",M8Y,43.636258,-79.498509


We now have the coordinates. Let's locate them on the map

In [5]:
address = 'Toronto,Ontario,Canada'
#!pip install Nominatim
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
toronto_map = folium.Map(location = [latitude,longitude], zoom_start=14)

for lat,lon,name in zip(df_coord['Latitude'],df_coord['Longitude'],df_coord['Borough']): 
 folium.CircleMarker(location=[lat,lon],popup=name,tooltip='Click here to see popup', fill = True, fill_corlor = 'blue').add_to(toronto_map)
toronto_map

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


We have located our neighborhood. Now let's use Foursquare to identify hospitals

In [6]:
CLIENT_ID = 'H2PLQIRRQS1EXEXKRSXFCU33A2UIFAP3AS5SXP4KCSDIA4N0' # your Foursquare ID
CLIENT_SECRET = 'SHXBOKPUG0Q24HDXLGRKU50C0GDVLMTPMHRY12FY4GRRANFU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Category ID']
    
    return(nearby_venues)
df_venues = getNearbyVenues(df_coord['Neighborhood'],df_coord['Latitude'],df_coord['Longitude'])


Your credentails:
CLIENT_ID: H2PLQIRRQS1EXEXKRSXFCU33A2UIFAP3AS5SXP4KCSDIA4N0
CLIENT_SECRET:SHXBOKPUG0Q24HDXLGRKU50C0GDVLMTPMHRY12FY4GRRANFU
Parkwoods
Victoria Village
Regent Park,Harbourfront
Lawrence Manor,Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern,Rouge
Don MillsNorth
Parkview Hill,Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park,Princess Gardens,Martin Grove,Islington,Cloverdale
Rouge Hill,Port Union,Highland Creek
Don MillsSouth
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate,Bloordale Gardens,Old Burnhamthorpe,Markland Wood
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Wilson Heights,Downsview North
Thorncliffe Park
Richmond,Adelaide,King
Dufferin,Dovercourt Village
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
The Danforth  East
Harbourfront East,Union Station,Toronto Island

## Analysis <a name="Analysis"></a>

In [7]:
df_venues['Venue'].nunique()


1251

There are 1251 venues in 0.5km near the Toronto center

We get the categories, let's filter out hospitals

In [8]:
df_hospitals = df_venues[df_venues['Category ID']=='4bf58dd8d48988d196941735']
df_hospitals

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Category ID
1585,"Kensington Market,Chinatown,Grange Park",43.653206,-79.400049,Toronto Western Hospital,43.653434,-79.406074,Hospital,4bf58dd8d48988d196941735


Seems like there is only one hospital in Toronto neighborhood, let's locate them on the map

In [9]:
#for lat,lon in zip(lo,df_hospitals['Venue Longitude']):
folium.Marker(location = [43.653434,-79.406074],
                      #radius = 10,
                      #color = 'red',
                      #fill = True,
                      #fill_color = '3186cc',
                      ).add_to(toronto_map)
toronto_map

Our main decision making is now moved to the population of the neighborhoods. Let's fetch population data from another source

In [10]:
link_pop ='https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods'
html_data_pop = requests.get(link_pop).text
soup = BeautifulSoup(html_data_pop,'html5lib')
tables = soup.find_all('table')         
#for i in range(0,len(tables)):
#  print("----------------- table",i,'-----------------')
#tables[1].td.td

df_neighbor = pd.DataFrame()
for i in range(1,len(tables)-4):
  data= pd.read_html(str(tables[i]),flavor = 'bs4')
  df_neighbor = df_neighbor.append(data[0])
df_neighbor

,Name,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map
0,Crescent Town,EY,0190.01,8157,0.40,20393,-10.0,23021,24.5,20.3,Bengali (18.1%),18.1% Bengali,NaN
1,Governor's Bridge/Bennington Heights,EY,0186.00,2112,1.87,1129,4.0,129904,7.1,13.3,Polish (1.4%),01.4% Polish,NaN
2,Leaside,EY,"0195.00, 0196.00",13876,2.81,4938,3.0,82670,9.7,10.5,Bulgarian (0.4%),00.4% Bulgarian,NaN
3,O'Connor–Parkview,EY,"0189.00, 0190.02, 0191.00, 0192.00, 0193.00",17740,4.94,3591,-6.1,33517,15.8,19.4,Urdu (3.2%),03.2% Urdu,NaN
4,Old East York,EY,"0180.00, 0181.01, 0181.02, 0182.00, 0183.00, 0...",52220,7.94,6577,-4.6,33172,22.0,19.1,Greek (4.3%),04.3% Greek,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Woburn,S,"0356.00, 0357.01, 0357.02, 0363.07, 0364.01, 0...",48507,13.34,3636,-1.5,26190,13.3,16.0,Gujarati (9.1%),09.1% Gujarati,NaN
152,Wychwood,OCoT,0116.00,4182,0.68,6150,-2.0,53613,17.1,20.1,Portuguese (2.7%),02.7% Portuguese,NaN
153,York Mills,NY,"0273.01, 0273.02, 0274.01, 0274.02",17564,7.29,2409,2.0,92099,10.0,11.8,Korean (4.0%),04.0% Korean,NaN
154,York University Heights,NY,"0311.02, 0311.03, 0311.04, 0311.05, 0311.06",26140,13.21,1979,-1.2,24432,15.2,20.4,Italian (6.6%),06.6% Italian,NaN


Let's map the population with postal code for each neighborhood combinations

In [11]:
df_neighbor_split = df_neighbor[['Name','Population']]
df_neighbor_split = df_neighbor_split.assign(var1 = df_neighbor_split['Name'].str.split('/')).explode('var1')

df_coord_pop = (df_coord.assign(var1=df_coord['Neighborhood'].str.split(',')).explode('var1').merge(
    df_neighbor_split,
    left_on = 'var1',
    right_on = 'var1',
    how = 'left')).dropna()
df_coord_pop

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,var1,Name,Population
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,Parkwoods,Parkwoods,26533
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,Victoria Village,Victoria Village,17047
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",M5A,43.654260,-79.360636,Regent Park,Regent Park/Trefann Court,10387
4,M6A,North York,"Lawrence Manor,Lawrence Heights",M6A,43.718518,-79.464763,Lawrence Manor,Lawrence Manor,13750
5,M6A,North York,"Lawrence Manor,Lawrence Heights",M6A,43.718518,-79.464763,Lawrence Heights,Lawrence Heights,3769
...,...,...,...,...,...,...,...,...,...
194,M4X,Downtown Toronto,"St. James Town,Cabbagetown",M4X,43.667967,-79.367675,St. James Town,St. James Town,14666
195,M4X,Downtown Toronto,"St. James Town,Cabbagetown",M4X,43.667967,-79.367675,Cabbagetown,Cabbagetown,11120
198,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",M8X,43.653654,-79.506944,The Kingsway,The Kingsway,8780
201,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160,Church and Wellesley,Church and Wellesley,13397


Now use K-means to cluster based on population

In [13]:
df0 = df_coord_pop.groupby(['Postal Code','Latitude','Longitude','Neighborhood'])['Population'].sum().reset_index()
df0

,Postal Code,Latitude,Longitude,Neighborhood,Population
0,M1B,43.806686,-79.194353,"Malvern,Rouge",67048.0
1,M1C,43.784535,-79.160497,"Rouge Hill,Port Union,Highland Creek",36470.0
2,M1E,43.763573,-79.188711,"Guildwood,Morningside,West Hill",49924.0
3,M1G,43.770992,-79.216917,Woburn,48507.0
4,M1J,43.744734,-79.239476,Scarborough Village,12796.0
5,M1K,43.727929,-79.262029,"Kennedy Park,Ionview,East Birchmount Park",13025.0
6,M1L,43.711112,-79.284577,"Golden Mile,Clairlea,Oakridge",24472.0
7,M1M,43.716316,-79.239476,"Cliffside,Cliffcrest,Scarborough Village West",23917.0
8,M1N,43.692657,-79.264848,"Birch Cliff,Cliffside West",12266.0
9,M1P,43.757410,-79.273304,"Dorset Park,Wexford Heights,Scarborough Town C...",14189.0


In [19]:
!pip install sklearn
from sklearn.cluster  import KMeans
kmeans = KMeans(n_clusters= 5,random_state = 4).fit(df0[['Population']])
df0['Cluster'] = kmeans.labels_

In [23]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# set 5 clusters
# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df0['Latitude'], df0['Longitude'], df0['Neighborhood'], df0['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map)
       
toronto_map

## Conclusion <a name = "Conclusion"> </a> 

Based on the clustering, we can place new hospital in area with red dot (highly populated areas)

Propose with the investors and stake holders to further decide the locations for the new hospital. Other elements should be taken into consideration are budget, time, and number of covid-19 patients currently have. 